In [67]:
from dotenv import load_dotenv
load_dotenv()

True

In [81]:
from pydantic import BaseModel,Field
from typing import List,Annotated,Optional
import operator

class User(BaseModel):
    name : str = Field(description="name of the organziation")
    logo : str = Field(description="Logo of the given website")
    description : str = Field(description='A detailed description of what the organization does ')
    services: List[str] = Field(description="A list of services offered by the organization on the given website")


class State(BaseModel):
    buyer : Optional[User] = Field(description=' ')
    seller :Optional[User] = Field(description=' ')
    client_requirement: str  
    additional_info : List[str]
    service_dept : str
    sections : List[str] = Field(description="Total sections inside the sales proposal")
    final_result : str = Field(description='')
    

In [82]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash')

In [83]:
def clean_to_list(result:str) :
    result = result.strip()
    if result.startswith('```python'):
        result = result[len('```python'):].strip()
    elif result.startswith('```json'):
        result = result[len('```json'):].strip()
    elif result.startswith('```'):
        result = result[len('```'):].strip()
    if result.endswith('```'):
        result = result[:-3].strip()
    return result

In [84]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# from ..utils import clean_to_list

template = '''You are a business consultant specialized in drafting professional sales proposals.

Based on the following services offered by the client, generate a list of high-level sections that should be included in a compelling business proposal. These sections should be relevant to the domain and type of services offered.

### Services:
{services}

Return only a clean list of section titles, no explanations or numbering.
'''
prompt = ChatPromptTemplate.from_template(template)


chain = prompt | llm | StrOutputParser()




def create_sections(state:State):
    result = chain.invoke({'services':state.service_dept})
    print(result)
    return {'sections':clean_to_list(result)}


In [102]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

p_template = '''You are an expert business proposal writer with decades of experience crafting high-value enterprise sales proposals.

Your task is to generate a deeply detailed, structured, and persuasive **Sales Proposal** for the following buyer. The proposal should be written with strategic insight, full use of available inputs, and customized as per the buyer's organization and the provided client requirements.

---

## 🧩 Buyer Organization Description:
{description_b}

---

## 🏢 Seller Organization Description:
{description_s}

---

## 📋 Client Requirements, Needs, and Context:
{client_requirements}

These may include expectations, project timelines, technical needs, business goals, constraints, or key deliverables. Every section of the proposal should reference and incorporate these points wherever relevant.

---

## 🏢 Target Department:
{service_dept}

Write the proposal keeping in mind this is the core department being addressed. Tailor the tone, terminology, and problem-solution framing to match this department’s priorities.

---

## 📚 Proposal Structure:
You must write the full proposal using the following sections, in order. Each section should be clearly titled.

{section_list}

---

### ✍️ Detailed Writing Guidelines:

- Write **comprehensive, high-depth content** for each section, reflecting its importance. For example, “Executive Summary” and “Proposed Solution” should be longer and more detailed than “About Us” or “Terms & Conditions”.
- Use **formal business language**, but keep it **persuasive and engaging**.
- **Use all inputs contextually** — reference buyer’s goals, timeline, constraints, and domain (from `client_requirements`) where they naturally fit.
- Include **metrics, technical insights, and examples** wherever appropriate.
- Assume this is going to an executive decision-maker: be **clear, convincing, and confident**.
- For sections like “Timeline”, “Team”, or “Implementation Plan”, be structured and precise.
- The result must be returned as a **list of dictionaries**, one per section, in this exact format:
  
  ```json
  [
    {{"Executive Summary": "Detailed content..."}},
    {{"Proposed Solution": "Detailed content..."}},
    ...
  ]

'''

# Set up LangChain prompt + parser
p_prompt = ChatPromptTemplate.from_template(p_template)
p_chain = p_prompt | llm | StrOutputParser()

# Function to generate the full proposal
def write_sales_proposal(state):
    section_string = "\n".join([f"- {s}" for s in state.sections])
    client_reqs = "\n".join(state.additional_info)

    result = p_chain.invoke({
        'description_s': state.seller.description,
        'description_b': state.buyer.description,
        'client_requirements': client_reqs,
        'service_dept': state.service_dept,
        'section_list': section_string
    })

    return {'final_result': result}


In [98]:
from langgraph.graph import StateGraph,START,END

builder = StateGraph(State)
builder.add_node(create_sections)
builder.add_node(write_sales_proposal)

builder.add_edge(START,'create_sections')
builder.add_edge('create_sections','write_sales_proposal')
builder.add_edge('write_sales_proposal',END)

In [99]:
graph = builder.compile()

In [88]:
buyer = User(name='GROWTHSUTRA' ,logo='https://static.wixstatic.com/media/cb6b3d_5c8f2b020ebe48b69bc8c163cc480156~mv2.png/v1/fill/w_60,h_60,al_c,q_85,usm_0.66_1.00_0.01,enc_avif,quality_auto/GrowthSutra%20Logo.png' ,description="At GrowthSutra, we are the go-to experts dedicated to accelerate brand and revenue growth for startups and SMBs. We provide the Fortune 500-caliber strategic thinking and flawless execution needed to gain market access, customer traction, and investor interest. Our team is comprised of seasoned marketing, communications, sales, and leadership executives, each with over 20+ years of real-world expertise launching and scaling disruptive brands across technology, e-commerce, climate, FMCG, and other major industries. We combine our team\\'s unmatched experience with rigorous project governance and proven data-driven frameworks to get you measurable results fast. Our approach is tailored to each client\\'s unique needs and objectives." ,services=['XPRT Co-Pilots', 'On Demand CMO', 'Ice Breakers', 'GTM Partner', 'Go-To-Market Partner', 'Revenue Architect', 'On-Demand CMO'])

In [89]:
seller = User(name = 'TCS', logo = 'https://www.tcs.com/content/dam/global-tcs/en/images/home/tata-logo-1.svg',description= 'TCS works hand in hand with world-leading investors.', services=  ['Artificial Intelligence', 'Cloud', 'Cognitive Business Operations', 'Consulting', 'Cybersecurity', 'Data & Analytics', 'Enterprise' ])

In [90]:
client_requirement = "Building a website for their domain"

additonal_info = ["This task is done in like 2 weeeks with cost of $4000 "]

service_dept = 'Technology'

In [100]:
state = State(buyer = buyer,seller = seller,client_requirement=client_requirement,additional_info=additonal_info,service_dept=service_dept,sections=[],final_result='')

In [79]:
create_sections(state)

Executive Summary
Introduction to [Client Company Name]
Understanding Your Technology Needs
Proposed Solutions
Technology Stack and Architecture
Implementation Plan
Project Timeline and Milestones
Pricing and Payment Terms
Service Level Agreements (SLAs)
Maintenance and Support
Case Studies/Success Stories
Team Expertise
Client Testimonials
Call to Action
Appendix (Optional)


{'sections': 'Executive Summary\nIntroduction to [Client Company Name]\nUnderstanding Your Technology Needs\nProposed Solutions\nTechnology Stack and Architecture\nImplementation Plan\nProject Timeline and Milestones\nPricing and Payment Terms\nService Level Agreements (SLAs)\nMaintenance and Support\nCase Studies/Success Stories\nTeam Expertise\nClient Testimonials\nCall to Action\nAppendix (Optional)'}

In [103]:
state = graph.invoke(state)

Executive Summary
Introduction to [Client Company Name]
Understanding Your Technology Needs
Proposed Solutions and Technologies
Implementation Plan & Timeline
Project Team & Expertise
Pricing and Payment Terms
Case Studies & Success Stories
Security and Compliance
Maintenance and Support
Next Steps & Call to Action
Appendix (Optional)


In [104]:
print(state['final_result'])

```json
[
  {
    "Executive Summary": "GrowthSutra, a leading brand and revenue growth accelerator for startups and SMBs, requires a robust technological solution to enhance its operational efficiency and data-driven decision-making.  This proposal outlines a tailored solution from TCS, delivered within two weeks at a cost of $4,000, focusing on enhancing GrowthSutra's technology infrastructure to support its rapid growth trajectory.  Our solution leverages TCS's expertise in working with world-leading investors to provide scalable and reliable technology, enabling GrowthSutra to maintain its high standard of Fortune 500-caliber strategic thinking and flawless execution. This proposal details the specific technological needs, the proposed solution, implementation plan, and the expert team dedicated to this project."
  },
  {
    "Introduction to GrowthSutra": "GrowthSutra is a dynamic organization specializing in accelerating brand and revenue growth for startups and SMBs.  Their prov